In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [4]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [5]:
os.getcwd()
train_data=pd.read_csv(r"/kaggle/input/fashionmnist/fashion-mnist_train.csv")
test_data=pd.read_csv(r"/kaggle/input/fashionmnist/fashion-mnist_test.csv")

In [6]:
print(train_data.head())
print(train_data.shape)     #(60000,785) we have 6000 data with 784(28*28) pixels
print(test_data.shape)     #(10000,785) we have 6000 data with 784(28*28) pixels
## AS data present in Pixels CSV , so we need to preprocess data

In [7]:
## Adding label names
class_label=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

## CSV preprocessing
train_label=train_data["label"]
test_label=test_data["label"]

# drop labels from dataset
train=train_data.drop("label",axis=1)
test=test_data.drop("label",axis=1)

# convert data to nd-array
train=train.to_numpy()
test=test.to_numpy()

train_label=train_label.to_numpy()
test_label=test_label.to_numpy()

In [8]:
sns.countplot(train_data["label"])  # each class have 6000 data points
sns.countplot(test_data["label"])   # each class have 1000 data points

In [9]:
## visualize train-image with labels
plt.figure(figsize=(20,20))
for i in range(20):   
    plt.subplot(4,5,i+1)    # subplot(in_row,in_col,i+1)    
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train[i].reshape(28,28),cmap="gray")
    label_class=train_label[i]
    plt.xlabel(class_label[label_class])   

In [10]:
## plot preprocessed specific-images
sample=30       ## index no.
plt.figure()
image=train[sample].reshape((28,28))
plt.grid(False)
plt.imshow(image,cmap="gray")
print(class_label[train_label[sample]]) 

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,Conv2D,MaxPool2D,Dropout

with tpu_strategy.scope():
    model=Sequential()
    model.add(Conv2D(filters=512,kernel_size=3,activation='relu',padding='same',input_shape=(28,28,1),name="INPUT_Layer"))
    model.add(MaxPool2D(2,2))
    model.add(Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',input_shape=(28,28,1)))
    model.add(MaxPool2D(2,2))
    model.add(Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',input_shape=(28,28,1)))
    model.add(MaxPool2D(2,2))
    model.add(Conv2D(filters=64,kernel_size=3,activation='relu',padding='same'))
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(10,activation='softmax',name="OUTPUT_Layer"))

    ## compile
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

model.summary()

In [12]:
### preprocessing train & test data

# As  our input shape given as "input_shape=(28,28,1)" but our data shape(60000,784)
train_reshaped=train.reshape(train.shape[0],28,28,1)
test_reshaped=test.reshape(test.shape[0],28,28,1)

## convert dtype to "float 32"
train_reshaped=train_reshaped.astype('float32')
test_reshaped=test_reshaped.astype('float32')

## Normalize data
train_reshaped/=255
test_reshaped/= 255

In [13]:
## Early Stpping
es=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)

In [14]:
## train
history=model.fit(train_reshaped,train_label,batch_size=1024,validation_split=0.3,validation_batch_size=1024,epochs=50,callbacks=[es])
# batch_size=1024(128*8) , validation_batch_size=1024(128*8)   

In [15]:
test_loss,test_accuracy=model.evaluate(test_reshaped,test_label,verbose="auto")
test_loss,test_accuracy

In [22]:
## plot model
tf.keras.utils.plot_model(model,to_file="/kaggle/working/model_architecture.png",show_shapes=True)

In [22]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [76]:
## Save model
os.chdir("/kaggle/working")
model.save('model_92.h5')